In [1]:
import tensorflow as tf
import numpy as np
from preprocess import *
import time

In [2]:
embedding_size = 300
training_epochs = 5
batch_size = 32
learning_rate = 0.001

In [3]:
train_data = read_data('data/train.txt')
train_data = np.array(train_data)
train_X = train_data[:,0]
train_Y = train_data[:,[-1]]

In [4]:
model = word2vec_load()

In [5]:
seq_length = [len(x) for x in train_X]
maxseq_length = max(seq_length)
maxseq_length = 100

In [6]:
X = tf.placeholder(tf.float32, shape=[None, maxseq_length * embedding_size])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [7]:
W = tf.Variable(tf.random_normal([maxseq_length * embedding_size, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [8]:
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

In [9]:
cost = -tf.reduce_mean(Y * tf.log(hypothesis + 1e-7) + (1 - Y) * tf.log(1 - hypothesis + 1e-7))
# cost_summ = tf.summary.scalar("cost", cost)

In [10]:
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [11]:
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
# acc_summ = tf.summary.scalar("accuracy", accuracy)

In [12]:
with tf.Session() as sess:    
    total_batch = int(len(train_X) / batch_size)
    save_path = './saved/model'
    saver = tf.train.Saver()

#     merged_summary = tf.summary.merge_all()
#     writer = tf.summary.FileWriter("./logs/logistic")
#     writer.add_graph(sess.graph)  # Show the graph
    
    start_time = time.time()

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):

        avg_loss = 0
        for step in range(total_batch):
            n = step * batch_size
            train_batch_X = train_X[n : n + batch_size]
            train_batch_X = [tokenize(row) for row in train_batch_X]
            train_batch_X = [embedding(model, row) for row in train_batch_X]
            train_batch_X = [zero_padding(maxseq_length, row, embedding_size) for row in train_batch_X]
            train_batch_Y = train_Y[n : n + batch_size]
            
            cost_val, _ = sess.run([cost, train], feed_dict={X: train_batch_X, Y: train_batch_Y})
            avg_loss += cost_val
            acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y})
            print('Batch : ', step + 1, '/', total_batch, '(epoch:', epoch, ')', 
                  ', BCE in this minibatch: ', cost_val, 'accuracy: ', float(acc))

        print('epoch:', epoch, ' train_loss:', float(avg_loss/total_batch))

#         summary = sess.run(merged_summary, feed_dict={X: train_X, Y: train_Y})
#         writer.add_summary(summary, global_step=epoch)

#         acc = sess.run(accuracy , feed_dict={X: train_X, Y: train_Y})
#         print('epoch:', epoch, ' train_loss:', float(avg_loss/total_batch), ' accuracy: ', float(acc))
        saver.save(sess, save_path, epoch)
        
    duration = time.time() - start_time
    minute = int(duration / 60)
    print('learning finish, minutes:', minute)


Batch :  1 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.6107376 accuracy:  0.65625
Batch :  2 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.1642897 accuracy:  0.625
Batch :  3 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.986195 accuracy:  0.625
Batch :  4 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.3144925 accuracy:  0.625
Batch :  5 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.9853737 accuracy:  0.59375
Batch :  6 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.8958867 accuracy:  0.5625
Batch :  7 / 5139 (epoch: 0 ) , BCE in this minibatch:  4.5437775 accuracy:  0.4375
Batch :  8 / 5139 (epoch: 0 ) , BCE in this minibatch:  5.777357 accuracy:  0.46875
Batch :  9 / 5139 (epoch: 0 ) , BCE in this minibatch:  4.064223 accuracy:  0.53125
Batch :  10 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.7449272 accuracy:  0.6875
Batch :  11 / 5139 (epoch: 0 ) , BCE in this minibatch:  4.7313056 accuracy:  0.59375
Batch :  12 / 5139 (epoch: 0 ) , BCE in this minibatch:  4.1101894 accuracy

Batch :  103 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.9908292 accuracy:  0.59375
Batch :  104 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.2360444 accuracy:  0.65625
Batch :  105 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.2038765 accuracy:  0.5
Batch :  106 / 5139 (epoch: 0 ) , BCE in this minibatch:  4.105401 accuracy:  0.5625
Batch :  107 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.9034085 accuracy:  0.46875
Batch :  108 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.5929756 accuracy:  0.53125
Batch :  109 / 5139 (epoch: 0 ) , BCE in this minibatch:  5.7110844 accuracy:  0.4375
Batch :  110 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.8563328 accuracy:  0.53125
Batch :  111 / 5139 (epoch: 0 ) , BCE in this minibatch:  1.3760833 accuracy:  0.8125
Batch :  112 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.6877801 accuracy:  0.53125
Batch :  113 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.63387 accuracy:  0.46875
Batch :  114 / 5139 (epoch: 0 ) , BCE in this minibat

Batch :  199 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.9975944 accuracy:  0.5625
Batch :  200 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.3471303 accuracy:  0.65625
Batch :  201 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.0296187 accuracy:  0.5
Batch :  202 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.771246 accuracy:  0.5625
Batch :  203 / 5139 (epoch: 0 ) , BCE in this minibatch:  1.665951 accuracy:  0.65625
Batch :  204 / 5139 (epoch: 0 ) , BCE in this minibatch:  4.6056175 accuracy:  0.5
Batch :  205 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.4832106 accuracy:  0.625
Batch :  206 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.6739867 accuracy:  0.65625
Batch :  207 / 5139 (epoch: 0 ) , BCE in this minibatch:  4.556394 accuracy:  0.53125
Batch :  208 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.652006 accuracy:  0.5
Batch :  209 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.2811704 accuracy:  0.59375
Batch :  210 / 5139 (epoch: 0 ) , BCE in this minibatch:  4.852

Batch :  300 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.8251781 accuracy:  0.625
Batch :  301 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.9774399 accuracy:  0.59375
Batch :  302 / 5139 (epoch: 0 ) , BCE in this minibatch:  5.420437 accuracy:  0.4375
Batch :  303 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.0460157 accuracy:  0.65625
Batch :  304 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.5870059 accuracy:  0.65625
Batch :  305 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.0576847 accuracy:  0.5625
Batch :  306 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.8529778 accuracy:  0.65625
Batch :  307 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.1638384 accuracy:  0.5
Batch :  308 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.681975 accuracy:  0.53125
Batch :  309 / 5139 (epoch: 0 ) , BCE in this minibatch:  4.486785 accuracy:  0.53125
Batch :  310 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.117293 accuracy:  0.5625
Batch :  311 / 5139 (epoch: 0 ) , BCE in this minibatch:

Batch :  399 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.870974 accuracy:  0.5
Batch :  400 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.2013373 accuracy:  0.5
Batch :  401 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.8857672 accuracy:  0.53125
Batch :  402 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.0826182 accuracy:  0.65625
Batch :  403 / 5139 (epoch: 0 ) , BCE in this minibatch:  1.2928969 accuracy:  0.75
Batch :  404 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.9820065 accuracy:  0.625
Batch :  405 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.4465194 accuracy:  0.65625
Batch :  406 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.7509422 accuracy:  0.6875
Batch :  407 / 5139 (epoch: 0 ) , BCE in this minibatch:  2.8722658 accuracy:  0.65625
Batch :  408 / 5139 (epoch: 0 ) , BCE in this minibatch:  3.5223064 accuracy:  0.625
Batch :  409 / 5139 (epoch: 0 ) , BCE in this minibatch:  5.3140745 accuracy:  0.4375
Batch :  410 / 5139 (epoch: 0 ) , BCE in this minibatch:  4.1

...

Batch :  4741 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.71036506 accuracy:  0.9375
Batch :  4742 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.44565967 accuracy:  0.90625
Batch :  4743 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.46294954 accuracy:  0.8125
Batch :  4744 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.326132 accuracy:  0.84375
Batch :  4745 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.14011809 accuracy:  0.96875
Batch :  4746 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.32535362 accuracy:  0.90625
Batch :  4747 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.43299913 accuracy:  0.84375
Batch :  4748 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.69259727 accuracy:  0.8125
Batch :  4749 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.0209926 accuracy:  0.8125
Batch :  4750 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.66935253 accuracy:  0.875
Batch :  4751 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.7375982 accuracy:  0.65625
Batch :  4752 / 5139 (epoch: 4 

Batch :  4839 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.0244259 accuracy:  0.6875
Batch :  4840 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.8482045 accuracy:  0.8125
Batch :  4841 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.8109337 accuracy:  0.875
Batch :  4842 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.1807717 accuracy:  0.75
Batch :  4843 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.6969826 accuracy:  0.84375
Batch :  4844 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.7156831 accuracy:  0.90625
Batch :  4845 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.0107317 accuracy:  0.84375
Batch :  4846 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.91010207 accuracy:  0.75
Batch :  4847 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.5015445 accuracy:  0.78125
Batch :  4848 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.36665618 accuracy:  0.875
Batch :  4849 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.93829536 accuracy:  0.75
Batch :  4850 / 5139 (epoch: 4 ) , BCE in th

Batch :  4938 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.3564198 accuracy:  0.6875
Batch :  4939 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.1521099 accuracy:  0.84375
Batch :  4940 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.38065678 accuracy:  0.84375
Batch :  4941 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.7770182 accuracy:  0.84375
Batch :  4942 / 5139 (epoch: 4 ) , BCE in this minibatch:  2.184401 accuracy:  0.65625
Batch :  4943 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.8108048 accuracy:  0.78125
Batch :  4944 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.33575332 accuracy:  0.8125
Batch :  4945 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.8082721 accuracy:  0.75
Batch :  4946 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.25221568 accuracy:  0.9375
Batch :  4947 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.2132237 accuracy:  0.78125
Batch :  4948 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.92857206 accuracy:  0.75
Batch :  4949 / 5139 (epoch: 4 ) , BCE

Batch :  5033 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.6202216 accuracy:  0.8125
Batch :  5034 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.73293114 accuracy:  0.8125
Batch :  5035 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.8880181 accuracy:  0.84375
Batch :  5036 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.9444661 accuracy:  0.84375
Batch :  5037 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.5448884 accuracy:  0.84375
Batch :  5038 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.6836732 accuracy:  0.875
Batch :  5039 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.3903043 accuracy:  0.78125
Batch :  5040 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.1024064 accuracy:  0.78125
Batch :  5041 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.9514406 accuracy:  0.78125
Batch :  5042 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.2765515 accuracy:  0.96875
Batch :  5043 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.057637 accuracy:  0.84375
Batch :  5044 / 5139 (epoch: 4 ) , B

Batch :  5127 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.0417166 accuracy:  0.75
Batch :  5128 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.21390656 accuracy:  0.875
Batch :  5129 / 5139 (epoch: 4 ) , BCE in this minibatch:  2.7618656 accuracy:  0.65625
Batch :  5130 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.42358696 accuracy:  0.90625
Batch :  5131 / 5139 (epoch: 4 ) , BCE in this minibatch:  2.1488426 accuracy:  0.78125
Batch :  5132 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.1053386 accuracy:  0.6875
Batch :  5133 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.2088759 accuracy:  0.8125
Batch :  5134 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.7530954 accuracy:  0.78125
Batch :  5135 / 5139 (epoch: 4 ) , BCE in this minibatch:  0.7640079 accuracy:  0.78125
Batch :  5136 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.2270511 accuracy:  0.78125
Batch :  5137 / 5139 (epoch: 4 ) , BCE in this minibatch:  1.1799366 accuracy:  0.8125
Batch :  5138 / 5139 (epoch: 4 ) , BCE